In [1]:

import pandas as pd

In [2]:

data = pd.read_csv('F:\kutty\severeinjury1.csv',encoding='cp1252')

In [3]:

import nltk

In [4]:
from nltk.tokenize import WhitespaceTokenizer

In [5]:
from nltk.stem.snowball import SnowballStemmer
stemming = SnowballStemmer("english")

In [17]:
data["narrative"]= data["summary.new"]+data["title.new"]

In [18]:

def identify_tokens(row):
    summary_title = row["narrative"]
    tokens = nltk.word_tokenize(summary_title)
    token_words = [ w for w in tokens if w.isalpha()]
    return token_words

In [19]:
data['words']= data.apply(identify_tokens,axis=1)

In [20]:
data['words']

0      [On, august, employees, and, of, templar, a, c...
1      [At, approximately, on, may, employee, a, fore...
2      [On, april, employee, was, installing, vinyl, ...
3      [On, november, employee, was, operating, an, m...
4      [At, approximately, on, september, employee, w...
5      [On, december, employee, a, carpenter, employe...
6      [On, august, employee, employed, by, perini, c...
7      [On, july, employee, with, aquatic, environmen...
8      [On, june, employee, employed, by, a, construc...
9      [On, june, an, employee, was, working, as, the...
10     [At, approximately, on, april, an, employee, w...
11     [At, on, january, employee, a, crane, operator...
12     [On, october, four, employees, were, abrasive,...
13     [On, march, employee, and, a, coworker, were, ...
14     [On, may, employee, was, performing, abrasive,...
15     [At, approximately, on, october, employee, was...
16     [Employee, was, sandblasting, inside, a, cubic...
17     [Employees, and, were, b

In [21]:
def stem_list(row):
    my_list = row['words']
    stemmed_list = [stemming.stem(word) for word in my_list]
    return(stemmed_list)

data['stemmed_words'] = data.apply(stem_list, axis=1)

In [22]:
data['stemmed_words']

0      [on, august, employe, and, of, templar, a, con...
1      [at, approxim, on, may, employe, a, foreman, r...
2      [on, april, employe, was, instal, vinyl, side,...
3      [on, novemb, employe, was, oper, an, machin, t...
4      [at, approxim, on, septemb, employe, was, work...
5      [on, decemb, employe, a, carpent, employ, by, ...
6      [on, august, employe, employ, by, perini, corp...
7      [on, juli, employe, with, aquat, environ, was,...
8      [on, june, employe, employ, by, a, construct, ...
9      [on, june, an, employe, was, work, as, the, dr...
10     [at, approxim, on, april, an, employe, was, wo...
11     [at, on, januari, employe, a, crane, oper, emp...
12     [on, octob, four, employe, were, abras, blast,...
13     [on, march, employe, and, a, cowork, were, do,...
14     [on, may, employe, was, perform, abras, blast,...
15     [at, approxim, on, octob, employe, was, attemp...
16     [employe, was, sandblast, insid, a, cubic, foo...
17     [employe, and, were, be,

In [23]:

from nltk.corpus import stopwords
stops = set(stopwords.words("english"))                  

def remove_stops(row):
    my_list = row['stemmed_words']
    meaningful_words = [w for w in my_list if not w in stops]
    return (meaningful_words)

data['stem_meaningful'] = data.apply(remove_stops, axis=1)

In [24]:
data['stem_meaningful']

0      [august, employe, templar, construct, compani,...
1      [approxim, may, employe, foreman, regular, emp...
2      [april, employe, instal, vinyl, side, singl, s...
3      [novemb, employe, oper, machin, employe, work,...
4      [approxim, septemb, employe, work, cowork, job...
5      [decemb, employe, carpent, employ, valley, tri...
6      [august, employe, employ, perini, corpor, work...
7      [juli, employe, aquat, environ, perform, maint...
8      [june, employe, employ, construct, compani, ro...
9      [june, employe, work, driver, tractor, trailer...
10     [approxim, april, employe, work, tile, setter,...
11     [januari, employe, crane, oper, employ, concre...
12     [octob, four, employe, abras, blast, clean, mc...
13     [march, employe, cowork, abras, blast, separ, ...
14     [may, employe, perform, abras, blast, oper, ro...
15     [approxim, octob, employe, attempt, clear, abr...
16     [employe, sandblast, insid, cubic, foot, rail,...
17     [employe, hoist, ft, tel

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
import math

In [26]:

data['stem_meaningful'] = ["  ".join(review) for review in data['stem_meaningful'].values]

In [27]:
data['stem_meaningful']

0      august  employe  templar  construct  compani  ...
1      approxim  may  employe  foreman  regular  empl...
2      april  employe  instal  vinyl  side  singl  st...
3      novemb  employe  oper  machin  employe  work  ...
4      approxim  septemb  employe  work  cowork  job ...
5      decemb  employe  carpent  employ  valley  trin...
6      august  employe  employ  perini  corpor  work ...
7      juli  employe  aquat  environ  perform  mainte...
8      june  employe  employ  construct  compani  roo...
9      june  employe  work  driver  tractor  trailer ...
10     approxim  april  employe  work  tile  setter  ...
11     januari  employe  crane  oper  employ  concret...
12     octob  four  employe  abras  blast  clean  mck...
13     march  employe  cowork  abras  blast  separ  m...
14     may  employe  perform  abras  blast  oper  roo...
15     approxim  octob  employe  attempt  clear  abra...
16     employe  sandblast  insid  cubic  foot  rail  ...
17     employe  hoist  ft  tele

In [28]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
v=TfidfVectorizer()

In [29]:
x=v.fit_transform(data['stem_meaningful'])

In [30]:
x

<1000x4267 sparse matrix of type '<type 'numpy.float64'>'
	with 45033 stored elements in Compressed Sparse Row format>

In [31]:
first_vector_v = x[0]

In [32]:
df = pd.DataFrame(first_vector_v.T.todense(),index = v.get_feature_names(),columns = ["tfidf"])
df.sort_values(by=["tfidf"],ascending=False)

,tfidf
lane,0.369472
vehicl,0.318440
highway,0.212294
turn,0.204727
struck,0.186184
templar,0.177080
refus,0.167130
veer,0.160069
emerg,0.154442
fiber,0.150119
